In [1]:
%load_ext autoreload
%autoreload 2

import higlass
import higlass.client as hgc
import higlass.tilesets as hgti

In [2]:
import base64
import cooler
import numpy as np
import pandas as pd
from random import random

def bigbed_like(
    bedlike_filepath: str,
    chromsizes_filepath: str = None,
    uuid: str = None,
    aggregator: callable = np.mean,
    log_scale: bool = False,
    categories: dict = None
):
    TILE_SIZE = 1024
    chromsizes = pd.read_csv(
        chromsizes_filepath,
        sep = '\t',
        index_col = 0,
        usecols = [0, 1],
        names = [None, 'size'],
        header = None
    )
    cum_chromsizes = np.cumsum(chromsizes.values)
    min_tile_cover = np.ceil(np.sum(chromsizes) / TILE_SIZE)
    max_zoom = int(np.ceil(np.log2(min_tile_cover)))
    resolutions = [2 ** x for x in range(max_zoom + 1)][::-1]
    
    bedlike = pd.read_csv(
        bedlike_filepath,
        sep = '\t',
        index_col = None,
        usecols = [0, 1, 2, 3, 4],
        names = ['chrom', 'start', 'end', 'name', 'score'],
        header = None
    )
    
    dense = np.zeros(cum_chromsizes[-1])

    # Densify bed data for later downsampling
    k = 0
    if categories is None:
        for region in bedlike.iterrows():
            length = int(region[1]['end'] - region[1]['start'])
            dense[k : k + length] = region[1]['score']
            k += length
    else:
        for region in bedlike.iterrows():
            length = int(region[1]['end'] - region[1]['start'])
            try:
                dense[k : k + length] = categories[region[1]['name']]
            except KeyError:
                dense[k : k + length] = categories['__others__']
            k += length
            
    if log_scale:
        dense += 1
        dense = np.log(dense)
    
    def tileset_info(chromsizes):
        tileset_info = {
            "min_pos": [0],
            "max_pos": [TILE_SIZE * 2 ** max_zoom],
            "max_width": TILE_SIZE * 2 ** max_zoom,
            "tile_size": TILE_SIZE,
            "max_zoom": max_zoom,
        }
        return tileset_info
    
    def abs2genomic(chromsizes, start_pos, end_pos):
        abs_chrom_offsets = np.r_[0, cum_chromsizes]
        cid_lo, cid_hi = np.searchsorted(abs_chrom_offsets, [start_pos, end_pos], side="right") - 1
        rel_pos_lo = start_pos - abs_chrom_offsets[cid_lo]
        rel_pos_hi = end_pos - abs_chrom_offsets[cid_hi]
        start = rel_pos_lo
        for cid in range(cid_lo, cid_hi):
            yield cid, start, int(chromsizes.iloc[cid])
            start = 0
        yield cid_hi, start, rel_pos_hi
        
    def downsample(data, bins):
        dim = data.shape[0]

        assert(dim >= bins)
        
        # Downsampling factor
        factor = np.round(dim / bins)
        
        # Temporary dimension to support downsampling by an integer
        tmp_dim = int(bins * factor)
        diff = tmp_dim - dim
        
        left_pad = int(np.floor(np.abs(diff) / 2))
        right_pad = int(np.ceil(np.abs(diff) / 2))
        
        tmp = np.zeros(tmp_dim)

        if diff == 0:
            tmp = data
        elif diff > 0:
            tmp[:left_pad] = data[0]
            tmp[-right_pad:] = data[-1]
        else:
            tmp = data[left_pad:dim - right_pad]

        return aggregator(tmp.reshape((int(tmp_dim / factor), -1)), axis = 1)
        
    def fetch(chrom, start, end, bins):        
        # Downsample
        return downsample(dense[start:end], bins)
        
    
    def get_tile(zoom_level, start_pos, end_pos):
        binsize = resolutions[zoom_level]

        arrays = []
        for cid, start, end in abs2genomic(chromsizes, start_pos, end_pos):
            bins = int(np.ceil((end - start) / binsize))
            try:
                chrom = chromsizes.index[cid]
                clen = chromsizes.values[cid]

                x = fetch(chrom, start, end, bins)

                # drop the very last bin if it is smaller than the binsize
                if end == clen and clen % binsize != 0:
                    x = x[:-1]
            except IndexError:
                # beyond the range of the available chromosomes
                # probably means we've requested a range of absolute
                # coordinates that stretch beyond the end of the genome
                x = np.zeros(bins)

            arrays.append(x)

        return np.concatenate(arrays)
    
    def tiles(tile_ids):
        generated_tiles = []
        for tile_id in tile_ids:
            tile_id_parts = tile_id.split(".")
            tile_position = list(map(int, tile_id_parts[1:3]))
            zoom_level = tile_position[0]
            tile_pos = tile_position[1]

            tile_size = TILE_SIZE * 2 ** (max_zoom - zoom_level)
            start_pos = tile_pos * tile_size
            end_pos = start_pos + tile_size
            dense = get_tile(zoom_level, start_pos, end_pos)

            if len(dense):
                max_dense = max(dense)
                min_dense = min(dense)
            else:
                max_dense = 0
                min_dense = 0

            min_f16 = np.finfo("float16").min
            max_f16 = np.finfo("float16").max

            has_nan = len([d for d in dense if np.isnan(d)]) > 0

            if (
                not has_nan
                and max_dense > min_f16
                and max_dense < max_f16
                and min_dense > min_f16
                and min_dense < max_f16
            ):
                tile_value = {
                    "dense": base64.b64encode(dense.astype("float16")).decode(
                        "utf-8"
                    ),
                    "dtype": "float16",
                }
            else:
                tile_value = {
                    "dense": base64.b64encode(dense.astype("float32")).decode(
                        "utf-8"
                    ),
                    "dtype": "float32",
                }

            generated_tiles += [(tile_id, tile_value)]
        return generated_tiles

    return hgti.Tileset(
        tileset_info=lambda: tileset_info(chromsizes),
        tiles=lambda tids: tiles(tids),
        uuid=uuid,
    )

In [3]:
ts_sp_fc_signal = hgti.bigwig('../../../peax/experiments/data/simulated-fold-change-signal-spiky-peaks.bigWig')
ts_sp_chip_signal = hgti.bigwig('../../../peax/experiments/data/simulated-chip-signal-spiky-peaks.bigWig')
ts_sp_input_signal = hgti.bigwig('../../../peax/experiments/data/simulated-input-signal-spiky-peaks.bigWig')

ts_bp_fc_signal = hgti.bigwig('../../../peax/experiments/data/simulated-fold-change-signal.bigWig')
ts_bp_chip_signal = hgti.bigwig('../../../peax/experiments/data/simulated-chip-signal.bigWig')
ts_bp_input_signal = hgti.bigwig('../../../peax/experiments/data/simulated-input-signal.bigWig')

ts_feature_scores = bigbed_like(
    '../../../peax/experiments/data/simulated-features.bed',
    '../../../peax/experiments/data/simulated-genome-chrom-sizes.tsv',
    uuid = 'feature_scores',
    aggregator = np.max,
    log_scale = True
)
ts_features = bigbed_like(
    '../../../peax/experiments/data/simulated-features.bed',
    '../../../peax/experiments/data/simulated-genome-chrom-sizes.tsv',
    uuid = 'features',
    aggregator = np.max,
    categories = {
        "Background": 1.0,
        "Binding": 1000.0,
        "__others__": 0.0
    }
)

In [4]:
tr_top_axis = hgc.Track(track_type='top-axis', position='top')
tr_sp_fc_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_sp_fc_signal,
    position='top',
    height=48,
    options={
        'name': 'ChIP sim spiked peaks fc signal',
        'barFillColor': '#008ca8',
        'valueScaleMin': 0
    }
)
tr_sp_chip_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_sp_chip_signal,
    position='top',
    height=48,
    options={ 'name': 'ChIP sim spiked peaks chip signal', 'barFillColor': '#0064a8' }
)
tr_sp_input_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_sp_input_signal,
    position='top',
    height=48,
    options={ 'name': 'ChIP sim spiked peaks input signal', 'barFillColor': '#999999' }
)
tr_bp_fc_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_bp_fc_signal,
    position='top',
    height=48,
    options={
        'name': 'ChIP sim broad peaks fc signal',
        'barFillColor': '#a8004c',
        'valueScaleMin': 0
    }
)
tr_bp_chip_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_bp_chip_signal,
    position='top',
    height=48,
    options={ 'name': 'ChIP sim broad peaks chip signal', 'barFillColor': '#a8009f' }
)
tr_bp_input_signal = hgc.Track(
    'horizontal-bar',
    tileset=ts_bp_input_signal,
    position='top',
    height=48,
    options={ 'name': 'ChIP sim broad peaks input signal', 'barFillColor': '#999999' }
)
tr_chip_feature_scores = hgc.Track(
    'horizontal-bar',
    tileset=ts_feature_scores,
    position='top',
    height=24,
    options={
        "name": "ChIP sim binding scores (log)",
        "colorRange": [
            "#f2f2f2",
            "#f2f2f2",
            "#f2f2f2",
            "#f2f2f2",
            "#bac8e2",
            "#7494c5",
            "#0064a8"
        ],
        "labelColor": "#666666",
    }
)
tr_chip_features = hgc.Track(
    'horizontal-1d-heatmap',
    tileset=ts_features,
    position='top',
    height=16,
    options={
        "name": "ChIP sim features",
        "colorRange": [
            "#f2f2f2",
            "#0064a8"
        ],
        "labelColor": "#666666",
    }
)

In [5]:
display, _, _ = higlass.display([
    hgc.View(
        [
            tr_top_axis,
            tr_sp_fc_signal,
            tr_sp_chip_signal,
            tr_sp_input_signal,
            tr_bp_fc_signal,
            tr_bp_chip_signal,
            tr_bp_input_signal,
            tr_chip_feature_scores,
            tr_chip_features
        ],
        initialXDomain = [0, 1.2e7]
    )
])
display

self.diskcache_directory /tmp/higlass-python/dc True
starting fuse
self.diskcache_directory /tmp/hgflask/dc True
starting fuse
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'WyBTvCLLR763EIlgKcarmA', 'tracks': {'top': [{'ty…